# PuLPモジュールを用いた最大流問題の最適解の計算
## Pandasを使用しない場合
## PuLPモジュール読み込み

In [1]:
import pulp

## 点の集合，枝の集合，容量を定義
点（地点）の集合は点のリストとして定義する．
枝（地点間を結ぶ道路）は，枝(i,j)をタプル(i,j)で表し，そのリストとして枝の集合を定義する．
容量は枝(i,j)をキーとする辞書として定義する．
始点sと終点tもここで定義しておく．

In [2]:
V = [i+1 for i in range(7)]
E = [(1,2), (1,3), (2,1), (2,4), (2,5), (3,4), (3,6), (4,7), (5,7), (6,7), (7,4)]
U = {(1,2):3, (1,3):5, (2,1):5, (2,4):2, (2,5):2, (3,4):3, (3,6):3, (4,7):2, (5,7):3, (6,7):4, (7,4):3}
s = 1
t = 7

## 決定変数の定義
各枝の流量xは枝をキーとする辞書として定義する．
非負条件はlowBound=0，容量制約はupBound=U[e]で設定する．
ここでlowBount=, upBound=は省略可．
始点における正味の流出量zは別に定義する．

In [3]:
x = {e:pulp.LpVariable(f'x{e[0]}_{e[1]}', lowBound=0, upBound=U[e]) for e in E} # upBound=U[e] 容量制約
z = pulp.LpVariable('z')

## 問題の定義
目的関数はzで，これを最大化する．

In [4]:
p = pulp.LpProblem('最大流問題', sense=pulp.LpMaximize)
p += z, '目的関数　流量'

点iに入る枝の集合をinflow，出る枝の集合をoutflowとする．
各枝eを探索し，e[1] == i，すなわち枝eが(*,i)なら点iに入る枝なので，eをinflowに追加する．
inflow.append(e)はリストinflowにeを追加している．
e[0] == i，すなわち枝eが(i,*)なら点iから出る枝なので，eをoutflowに追加する．
inflowとoutflowが決まったら，出る枝に相当する変数の和から入る枝に相当する変数の和を引き，これをnet_outflowとする．
点iが始点sの場合は，net_outflowはz，点iが終点tの場合は，net_outflowは-z，点iが始点と終点以外の場合はnet_flowは0になる．

In [5]:
# 流量保存制約
for i in V:
    inflow = []
    outflow = []
    for e in E:
        if e[1] == i:
            inflow.append(e)
        if e[0] == i:
            outflow.append(e)
    net_outflow = pulp.lpSum(x[e] for e in outflow) - pulp.lpSum(x[e] for e in inflow)
    if i == s:
        p += net_outflow == z, f'始点{i}'
    elif i == t:
        p += net_outflow == -z, f'終点{i}'
    else:
        p += net_outflow == 0, f'点{i}'
p

最大流問題:
MAXIMIZE
1*z + 0
SUBJECT TO
始点1: x1_2 + x1_3 - x2_1 - z = 0

点2: - x1_2 + x2_1 + x2_4 + x2_5 = 0

点3: - x1_3 + x3_4 + x3_6 = 0

点4: - x2_4 - x3_4 + x4_7 - x7_4 = 0

点5: - x2_5 + x5_7 = 0

点6: - x3_6 + x6_7 = 0

終点7: - x4_7 - x5_7 - x6_7 + x7_4 + z = 0

VARIABLES
x1_2 <= 3 Continuous
x1_3 <= 5 Continuous
x2_1 <= 5 Continuous
x2_4 <= 2 Continuous
x2_5 <= 2 Continuous
x3_4 <= 3 Continuous
x3_6 <= 3 Continuous
x4_7 <= 2 Continuous
x5_7 <= 3 Continuous
x6_7 <= 4 Continuous
x7_4 <= 3 Continuous
z free Continuous

## 最適解の計算と計算結果の読み取り

In [6]:
result = p.solve()

In [7]:
pulp.LpStatus[result]

'Optimal'

In [8]:
pulp.value(p.objective)

7.0

In [9]:
for v in p.variables():
    if pulp.value(v) > 0:
        print(f'{v} = {pulp.value(v)}')

x1_2 = 2.0
x1_3 = 5.0
x2_5 = 2.0
x3_4 = 2.0
x3_6 = 3.0
x4_7 = 2.0
x5_7 = 2.0
x6_7 = 3.0
z = 7.0


最適解は，
x1_2 = 2, x1_3 = 5, x2_1 = 0, x2_4 = 0, x2_5 = 2, x3_4 = 2, x3_6 = 3, x4_7 = 2, x5_7 = 2, x6_7 = 3, x7_4 = 0, z= 7.0
で，始点から終点までは最大7流すことができる．

例えば，x1_2  = 3, x1_3 = 4 ということで，始点1から点2へは容量いっぱいの3流して，始点1から点3へは容量より1少ない4流している．始点1からの流出量は x1_2 + x1_3 = 7で，流入量はx2_1 = 0 である．すなわち，始点1の正味の流出量は7である．

終点7の流入量は x4_7 + x5_7 + x6_7 = 7 で，流出量は x7_4 = 0 である．すなわち，終点7の正味の流入量は7で，確かに，始点1の正味の流出量と終点7の正味の流入量が等しくなっている．